In [1]:
%matplotlib inline

# Construir la red neuronal

Las redes neuronales se componen de capas/módulos que realizan operaciones con datos. El espacio de nombres [``torch.nn``](https://pytorch.org/docs/stable/nn.html) proporciona todos los componentes básicos que necesita para construir su propia red neuronal. Cada módulo en PyTorch subclasifica el [``nn.Module``](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). Una red neuronal es un módulo en sí mismo que consta de otros módulos (capas). Esta estructura anidada permite construir y administrar arquitecturas complejas fácilmente.

En las siguientes secciones, crearemos una red neuronal para clasificar imágenes en el conjunto de datos FashionMNIST.

In [2]:
# import os
import torch
from torch import nn
# from torch.utils.data import DataLoader
# from torchvision import datasets, transforms

# Obtener el dispositivo para entrenamiento

Queremos poder entrenar nuestro modelo en un acelerador de hardware como la GPU, si está disponible. Revisemos para ver si [``torch.cuda``](https://pytorch.org/docs/stable/notes/cuda.html) está disponible, de lo contrario continuaremos usando la CPU.

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


# Definir la clase Red Neuronal

Definimos nuestra red neuronal mediante subclases ``nn.Modulee`` inicializamos las capas de la red neuronal en ``__init__``. Cada subclase ``nn.Module`` implementa las operaciones sobre los datos de entrada en el método ``forward``.

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()   # Se inicializa el módulo nn.Module
        self.flatten = nn.Flatten()             # Se crea una primera capa que aplana la imagen de entrada
        self.linear_relu_stack = nn.Sequential( # Se crea una arquitectura secuencial:
            nn.Linear(28*28, 512),                  # Se añade una primera capa lineal que está preparada para que le entre un vector de 28*28 (784)
                                                    # y sacará un vector de 512
            nn.ReLU(),                              # Se añade una no linealidad
            nn.Linear(512, 512),                    # Se añade una segunda capa lineal que le entran 512 datos y saca 512 datos
            nn.ReLU(),                              # Se añade una no linealidad
            nn.Linear(512, 10)                      # Se añade una tercera capa lineal que le entran 512 datos y saca un array de tamaño 10 (el número
                                                    # de etiquetas)
        )

    def forward(self, x):
        x = self.flatten(x)                         # Se pasa la imagen por la capa de aplanado para aplanar la imagen
        logits = self.linear_relu_stack(x)          # Se pasa el vector resultante por la red
        return logits

Creamos una instancia de ``NeuralNetwork``, la movemos al ``device`` e imprimimos su estructura.

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Para usar el modelo, le pasamos los datos de entrada. Esto ejecuta el modelo forward, junto con algunas [operaciones en segundo plano](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866). ¡No llames a ``model.forward()`` directamente!

Llamar al modelo en la entrada devuelve un tensor de 10 dimensiones con valores pronosticados sin procesar para cada clase. Obtenemos las probabilidades de predicción pasándolas a través de una instancia del módulo ``nn.Softmax``.

In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)

pred_probab = nn.Softmax(dim=1)(logits)
print(f"A la salida el modelo devuelve un vector de {pred_probab.shape[1]} dimensiones: {pred_probab}")

y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

A la salida el modelo devuelve un vector de 10 dimensiones: tensor([[0.1076, 0.1068, 0.0979, 0.1054, 0.0946, 0.1016, 0.0950, 0.0950, 0.0987,
         0.0974]], device='cuda:0', grad_fn=<SoftmaxBackward>)
Predicted class: tensor([0], device='cuda:0')


--------------




## Capas del modelo

Analicemos las capas en el modelo FashionMNIST. Para ilustrarlo, tomaremos un minibatch de muestra de 3 imágenes de tamaño 28x28 y veremos qué le pasa a medida que lo pasamos por la red.

In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

Inicializamos la capa [``nn.Flatten``](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) para convertir cada imagen 2D 28x28 en una matriz contigua de 784 valores de píxeles (se mantiene la dimensión del minibatch (en dim = 0)).

In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

La [capa lineal](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) es un módulo que aplica una transformación lineal en la entrada usando sus pesos y sesgos almacenados.

In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

Las activaciones no lineales son las que crean las complejas asignaciones entre las entradas y salidas del modelo. Se aplican después de transformaciones lineales para introducir la no linealidad, lo que ayuda a las redes neuronales a aprender una amplia variedad de fenómenos.

En este modelo, usamos [``nn.ReLU``](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) entre nuestras capas lineales, pero hay otras activaciones para introducir la no linealidad en su modelo.

In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3812, -0.3092, -0.9389, -0.1036,  0.0418, -0.7085, -0.3242,  0.2523,
         -0.2707, -0.2178,  0.2485,  0.2694, -0.2686,  0.5476, -0.0729, -0.4035,
          0.3148, -0.6317,  0.0987,  0.2874],
        [ 0.5722, -0.4652, -0.5701, -0.1610,  0.3348, -0.6676, -0.1398,  0.3676,
         -0.0971, -0.1017,  0.0411,  0.1108, -0.6608,  0.6363, -0.1903,  0.4054,
          0.0733, -0.1758,  0.2478,  0.1387],
        [ 0.2917, -0.4701, -0.5797,  0.0855,  0.1653, -0.5012,  0.0351,  0.2473,
         -0.1083,  0.0202, -0.1595,  0.1441, -0.4096,  0.5220, -0.1599, -0.1122,
          0.1384,  0.0906,  0.0486, -0.0508]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.3812, 0.0000, 0.0000, 0.0000, 0.0418, 0.0000, 0.0000, 0.2523, 0.0000,
         0.0000, 0.2485, 0.2694, 0.0000, 0.5476, 0.0000, 0.0000, 0.3148, 0.0000,
         0.0987, 0.2874],
        [0.5722, 0.0000, 0.0000, 0.0000, 0.3348, 0.0000, 0.0000, 0.3676, 0.0000,
         0.0000, 0.0411, 0.1108, 0.0000, 0.6363, 0.000

### nn.Sequential

[``nn.Sequential``](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) es un contenedor ordenado de módulos. Los datos se pasan a través de todos los módulos en el mismo orden definido. Puede utilizar contenedores secuenciales para armar una red rápida como ``seq_modules``.

In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

print(logits.shape)

torch.Size([3, 10])


### nn.Softmax

La última capa lineal de la red neuronal devuelve ``logits`` (valores brutos en [-inf, inf]) que se pasan al módulo [``nn.Softmax``](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html). Los ``logits`` se escalan a valores [0, 1] que representan las probabilidades predichas del modelo para cada clase. El parámetro ``dim`` indica la dimensión a lo largo de la cual los valores deben sumar 1.

In [13]:
print(f"Before softmax: {logits}\n\n")

softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

print(f"After softmax: {pred_probab}, tamaño softmax: {pred_probab.shape}")

Before softmax: tensor([[-0.2088,  0.0115,  0.2385,  0.2981, -0.2063, -0.2559, -0.2752, -0.2882,
         -0.0599, -0.0644],
        [-0.1546,  0.0353,  0.2467,  0.3408, -0.3545, -0.3311, -0.2543, -0.3207,
         -0.0808,  0.0916],
        [-0.2386,  0.1187,  0.2307,  0.1439, -0.2649, -0.2081, -0.2315, -0.3275,
          0.0764, -0.0397]], grad_fn=<AddmmBackward>)


After softmax: tensor([[0.0862, 0.1074, 0.1348, 0.1431, 0.0864, 0.0822, 0.0806, 0.0796, 0.1000,
         0.0996],
        [0.0900, 0.1088, 0.1345, 0.1477, 0.0737, 0.0755, 0.0815, 0.0762, 0.0969,
         0.1151],
        [0.0833, 0.1190, 0.1331, 0.1220, 0.0811, 0.0858, 0.0838, 0.0762, 0.1141,
         0.1016]], grad_fn=<SoftmaxBackward>), tamaño softmax: torch.Size([3, 10])


### Parámetros del modelo

Muchas capas dentro de una red neuronal están parametrizadas, es decir, tienen pesos y sesgos asociados que se optimizan durante el entrenamiento. La subclasificación ``nn.Module`` rastrea automáticamente todos los campos definidos dentro de su objeto de modelo y hace que todos los parámetros sean accesibles usando los métodos de su modelo ``parameters()`` o ``named_parameters()``.

En este ejemplo, iteramos sobre cada parámetro e imprimimos su tamaño y una vista previa de sus valores.

In [14]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0206,  0.0110,  0.0090,  ...,  0.0110, -0.0005,  0.0167],
        [-0.0214,  0.0210, -0.0219,  ...,  0.0284, -0.0220,  0.0099]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0013, -0.0295], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0328, -0.0439,  0.0011,  ..., -0.0175, -0.0339, -0.0433],
        [-0.0111,  0.0403,  0.0016,  ..., -0.0217, -0.0061,  0.0429]],
       device='cuda:0', grad_fn=<Sl